## Backtesting 

### Store Metrics/Plot Visualisations here

In [5]:
import sys
from pathlib import Path
import pandas as pd
import json
from datetime import datetime

sys.path.append('..')
from strategies.mean_reversion import MeanReversionStrat, Trade
from utilities.api_client import APIClient
from strategies.indicators import rsi, bb, atr, std_dev

# Load config
with open('../config.json', 'r') as f:
    config = json.load(f)

# Initialize backtest parameters
INITIAL_BALANCE_USDT = 10000  # Starting with 10k USDT
TRADING_FEES = 0.001  # 0.1% maker/taker fee on Binance



In [ ]:
def run(backtest: MeanReversionStrat):
    #make for loop until no more data left with update period function
    while backtest.env.get_current_data() is not None:
        


In [ ]:
class Backtest:
    def __init__(self, strategy, data, initial_balance):
        self.strategy = strategy
        self.data = data
        self.balance = initial_balance
        self.positions = []
        self.trades = []
        self.equity_curve = []
        
    def run(self):
        # Iterate through each day in the data
        for i in range(len(self.data)):
            current_data = self.data.iloc[:i+1]
            if len(current_data) < 20:  # Need enough data for indicators
                continue
                
            # Get signal from strategy
            signal = self.strategy.gen_signal(current_data)
            
            # Execute trades based on signal
            if signal != 0:
                # Calculate position size based on risk management
                size = self.strategy.pos_size
                
                # Include trading fees
                fee = size * current_data.iloc[-1]['close'] * TRADING_FEES
                
                # Execute trade
                self.strategy.execute_trade(signal, current_data, size)
                
                # Update balance
                self.balance -= fee
            
            # Record equity at this point
            total_equity = self.balance
            if self.strategy.pos != 0:
                # Add unrealized P&L if we have an open position
                current_price = current_data.iloc[-1]['close']
                total_equity += self.strategy.pos_size * current_price
                
            self.equity_curve.append({
                'timestamp': current_data.iloc[-1]['timestamp'],
                'equity': total_equity
            })
            
    def get_results(self):
        equity_df = pd.DataFrame(self.equity_curve)
        
        # Calculate basic metrics
        total_return = (equity_df['equity'].iloc[-1] - INITIAL_BALANCE_AUD) / INITIAL_BALANCE_AUD
        
        # Calculate drawdown
        equity_df['peak'] = equity_df['equity'].cummax()
        equity_df['drawdown'] = (equity_df['peak'] - equity_df['equity']) / equity_df['peak']
        max_drawdown = equity_df['drawdown'].max()
        
        return {
            'total_return': total_return,
            'max_drawdown': max_drawdown,
            'num_trades': len(self.strategy.trades),
            'equity_curve': equity_df
        }

# Fetch historical data
client = APIClient(config, "", "")  # Empty strings for API keys since we're just backtesting
historical_data = client.fetch_ohlcv_df('BTC/AUD')

# Initialize strategy
strategy = MeanReversionStrat(config=config, api_key="", api_secret="")

# Run backtest
backtest = Backtest(strategy, historical_data, INITIAL_BALANCE_AUD)
backtest.run()

# Get and display results
results = backtest.get_results()
print(f"Total Return: {results['total_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Number of Trades: {results['num_trades']}")

# Plot equity curve
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results['equity_curve']['timestamp'], results['equity_curve']['equity'])
plt.title('Equity Curve')
plt.xlabel('Date')
plt.ylabel('Portfolio Value (AUD)')
plt.grid(True)
plt.show()
